# Pull Sentinel-2 time series with Google Earth Engine

This notebook builds monthly Sentinel-2 composites over the pipe corridor mask.
- Input: `data/geometry/corridors.geojson`
- Output: monthly composites exported to Google Drive

Requirements: `earthengine-api` (install locally) and a GEE account.


In [ ]:
import json
import ee

# First run only
# ee.Authenticate()

ee.Initialize()


In [ ]:
# Load corridor mask from local GeoJSON
with open('data/geometry/corridors.geojson', 'r') as f:
    corridors_geojson = json.load(f)

corridors_fc = ee.FeatureCollection(corridors_geojson)
roi = corridors_fc.geometry().buffer(0)


In [ ]:
# Sentinel-2 Surface Reflectance (harmonized)
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

def mask_s2_sr(image):
    qa = image.select('QA60')
    cloud = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud).divide(10000)

def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def monthly_composites(start, end):
    start = ee.Date(start)
    end = ee.Date(end)
    n_months = end.difference(start, 'month')

    def by_month(n):
        n = ee.Number(n)
        month_start = start.advance(n, 'month')
        month_end = month_start.advance(1, 'month')
        col = (s2
            .filterDate(month_start, month_end)
            .filterBounds(roi)
            .map(mask_s2_sr)
            .map(add_indices))
        img = col.median().set({
            'system:time_start': month_start.millis(),
            'month': month_start.format('YYYY-MM')
        })
        return img

    return ee.ImageCollection(ee.List.sequence(0, n_months.subtract(1)).map(by_month))

monthly = monthly_composites('2018-01-01', '2024-01-01')


In [ ]:
# Export monthly composites to Google Drive
# Adjust folder name, dates, and bands as needed

folder = 'water_leakage_s2_monthly'
bands = ['B2', 'B3', 'B4', 'B8', 'NDVI']

imgs = monthly.toList(monthly.size())
n = monthly.size().getInfo()

for i in range(n):
    img = ee.Image(imgs.get(i))
    date_str = img.get('month').getInfo()
    task = ee.batch.Export.image.toDrive(
        image=img.select(bands),
        description=f's2_{date_str}',
        folder=folder,
        fileNamePrefix=f's2_{date_str}',
        region=roi.bounds(),
        scale=10,
        maxPixels=1e13
    )
    task.start()
    print('Started', date_str)


Optional: export per-corridor mean NDVI time series as a CSV.

```python
def monthly_stats(collection, fc):
    def per_img(img):
        stats = img.select(['NDVI']).reduceRegions(
            collection=fc, reducer=ee.Reducer.mean(), scale=10)
        return stats.map(lambda f: f.set('month', img.get('month')))
    return ee.FeatureCollection(collection.map(per_img).flatten())

stats_fc = monthly_stats(monthly, corridors_fc)

task = ee.batch.Export.table.toDrive(
    collection=stats_fc,
    description='s2_monthly_ndvi',
    folder='water_leakage_s2_monthly',
    fileNamePrefix='s2_monthly_ndvi',
    fileFormat='CSV'
)
task.start()
```
